<a href="https://colab.research.google.com/github/hissain/mlworks/blob/main/codes/Building_an_Agent_using_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build an Agent

This guide assumes familiarity with the following concepts:

- [Chat Models](https://python.langchain.com/docs/concepts/#chat-models)
- [Tools](https://python.langchain.com/docs/concepts/#tools)
- [Agents](https://python.langchain.com/docs/concepts/#agents)

By themselves, language models can't take actions - they just output text.
A big use case for LangChain is creating **agents**.
Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them.
After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

## End-to-end agent

The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the individual components and what each part does - but if you want to just grab some code and get started, feel free to use this!

In [ ]:
# Installation
!pip install -qU langchain-community langgraph tavily-python langgraph-checkpoint-sqlite
!pip install -qU langchain-openai
!pip install -qU langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 853.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58

In [ ]:
# Import relevant functionality
import os
import getpass
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [ ]:
# os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


In [ ]:
# model = ChatOpenAI(model="gpt-3.5-turbo")
model = ChatGroq(model="gemma2-9b-it")

In [ ]:
os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········


## Define tools

We first need to create the tools we want to use. Our main tool of choice will be [Tavily](https://tavily.com/) - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Bangladesh today?")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Dhaka', 'region': '', 'country': 'Bangladesh', 'lat': 23.72, 'lon': 90.41, 'tz_id': 'Asia/Dhaka', 'localtime_epoch': 1727779410, 'localtime': '2024-10-01 16:43'}, 'current': {'last_updated_epoch': 1727778600, 'last_updated': '2024-10-01 16:30', 'temp_c': 32.1, 'temp_f': 89.7, 'is_day': 1, 'condition': {'text': 'Light rain shower', 'icon': '//cdn.weatherapi.com/weather/64x64/day/353.png', 'code': 1240}, 'wind_mph': 4.9, 'wind_kph': 7.9, 'wind_degree': 253, 'wind_dir': 'WSW', 'pressure_mb': 1001.0, 'pressure_in': 29.57, 'precip_mm': 0.16, 'precip_in': 0.01, 'humidity': 65, 'cloud': 69, 'feelslike_c': 38.8, 'feelslike_f': 101.8, 'windchill_c': 32.1, 'windchill_f': 89.7, 'heatindex_c': 38.8, 'heatindex_f': 101.8, 'dewpoint_c': 24.7, 'dewpoint_f': 76.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 7.0, 'gust_mph': 7.7, 'gust_kph': 12.4}}"}, {'url': 'https://www.bssnews.net/news/213162', 'content': '01 Oct 2024, 10:56 Ligh

## Using Language Models

Next, let's learn how to use a language model by to call tools. LangChain supports many different language models that you can use interchangably - select the one you want to use below!



You can call the language model by passing in a list of messages. By default, the response is a `content` string.

In [ ]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! 👋  How can I help you today? 😊\n'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use `.bind_tools` to give the language model knowledge of these tools

In [ ]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the `content` field as well as the `tool_calls` field.

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Hi!'}, 'id': 'call_vvtz', 'type': 'tool_call'}]


Now, let's try calling it with some input that would expect a tool to be called.

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Bangladesh today?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Bangladesh today'}, 'id': 'call_t6x2', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

## Create the agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using [LangGraph](/docs/concepts/#langgraph) to construct the agent.
Currently we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.


Now, we can initialize the agent with the LLM and the tools.

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_react_agent` will call `.bind_tools` for us under the hood.

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

We can now run the agent on a few queries! Note that for now, these are all **stateless** queries (it won't remember previous interactions). Note that the agent will return the **final** state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='1f0870de-7a38-4f90-8288-009e0ba01c06'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4yg8', 'function': {'arguments': '{"query":"hi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 983, 'total_tokens': 1068, 'completion_time': 0.154545455, 'prompt_time': 0.03174423, 'queue_time': 0.0030977189999999988, 'total_time': 0.186289685}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-72774b60-6aa1-4a4e-8c20-6b22de3613cd-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'hi'}, 'id': 'call_4yg8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 983, 'output_tokens': 85, 'total_tokens': 1068}),
 ToolMessage(content="HTTPError('400 Client Error: Bad Request for url: https://api.tavily.com/search')"

In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)

Let's now try it out on an example where it should be invoking the tool

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Bangladesh today?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in Bangladesh today?', additional_kwargs={}, response_metadata={}, id='addb5d2a-4959-4e2a-a7d8-6a2fe35c9c03'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_e29b', 'function': {'arguments': '{"query":"weather in Bangladesh today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 988, 'total_tokens': 1078, 'completion_time': 0.163636364, 'prompt_time': 0.035156477, 'queue_time': 0.08520396899999999, 'total_time': 0.198792841}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-43d13376-231e-4bdd-8346-bc8684242bc0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Bangladesh today'}, 'id': 'call_e29b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 988, 'output_tokens': 90, 'total_tokens': 1078}),
 ToolMessage(content

## Streaming Messages

We've seen how the agent can be called with `.invoke` to get back a final response. If the agent is executing multiple steps, that may take a while. In order to show intermediate progress, we can stream back messages as they occur.

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Bangladesh today?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1j4p', 'function': {'arguments': '{"query":"weather in Bangladesh today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 988, 'total_tokens': 1078, 'completion_time': 0.163636364, 'prompt_time': 0.032016907, 'queue_time': 0.0030873460000000047, 'total_time': 0.195653271}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fb20b4ab-8a85-4dca-8088-fda3068402c5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Bangladesh today'}, 'id': 'call_1j4p', 'type': 'tool_call'}], usage_metadata={'input_tokens': 988, 'output_tokens': 90, 'total_tokens': 1078})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Dhaka\', \'regi

## Streaming tokens

In addition to streaming back messages, it is also useful to be streaming back tokens.
We can do this with the `.astream_events` method.

:::important
This `.astream_events` method only works with Python 3.11 or higher.
:::

In [ ]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in Bangladesh?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1727766680, \'localtime\': \'2024-10-01 00:11\'}, \'current\': {\'last_updated_epoch\': 1727766000, \'last_updated\': \'2024-10-01 00:00\', \'temp_c\': 20.2, \'temp_f\': 68.4, \'is_day\': 0, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 2.2, \'wind_kph\': 3.6, \'wind_degree\': 20, \'wind_dir\': \'NNE\', \'pressure_mb\': 1013.0, \'pressure_in\': 29.91, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 61, \'cloud\': 25, \'feelslike_c\': 20.2, \'feelslike_f\': 68.4, \'windchill_c\

## Adding in memory

As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a `thread_id` when invoking the agent (so it knows which thread/conversation to resume from).

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-597c963e-560e-4591-8151-066197c3b4f0-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96})]}}
----


In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 108, 'total_tokens': 122, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-add17659-e299-49f7-9e1e-ef513e749dc8-0', usage_metadata={'input_tokens': 108, 'output_tokens': 14, 'total_tokens': 122})]}}
----


Example [LangSmith trace](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

If I want to start a new conversation, all I have to do is change the `thread_id` used

In [ ]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I am unable to know your name as I don't have access to personal information. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 86, 'total_tokens': 111, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6bc05096-2803-4571-8d21-c4340dfaa372-0', usage_metadata={'input_tokens': 86, 'output_tokens': 25, 'total_tokens': 111})]}}
----


## Conclusion

That's a wrap! In this quick start we covered how to create a simple agent.
We've then shown how to stream back a response - not only the intermediate steps, but also tokens!
We've also added in memory so you can have a conversation with them.
Agents are a complex topic, and there's lot to learn!

For more information on Agents, please check out the [LangGraph](/docs/concepts/#langgraph) documentation. This has it's own set of concepts, tutorials, and how-to guides.